In [2]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

2024-02-21 14:35:30.120801: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-21 14:35:30.153241: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-21 14:35:30.983894: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-21 14:35:30.984972: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-21 14:35:35.973573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

#### Import neural network, transformers, base input values

In [28]:
# Import the saved neural network
nn_tau_em = load_model("/glade/work/wchuang/mlmicrophysics/tau_run_10/optimized/quantile_neural_net_keras.h5")

# Import the quantile transformers
with open("/glade/work/wchuang/mlmicrophysics/tau_run_10/optimized/output_quantile_transform.pkl", "rb") as out_quant_transform_file:
    output_scaler = pickle.load(out_quant_transform_file)
with open("/glade/work/wchuang/mlmicrophysics/tau_run_10/optimized/input_quantile_transform.pkl", "rb") as in_quant_transform_file:
    input_scaler = pickle.load(in_quant_transform_file)

# Read in the fortran-scam inputs
fortran_directory = "/glade/derecho/scratch/wchuang/scam_ml_port6_emulate8_optimized.arm97/run/"
fortran_input_file = fortran_directory + "test_input.dat"
input = np.fromfile(fortran_input_file, dtype=np.float64, sep=' ').reshape(1, -1)

# Set column names for inputs and outputs
input_col_names = ["qc", "qr", "nc", "nr", "pgam", "lamc", "lamr", "n0r", "rho_clubb"]
output_col_names = ["qctend", "nctend", "nrtend"]

View the various inputs/outputs

In [29]:
nn_tau_em.loss

<function keras.src.losses.huber(y_true, y_pred, delta=1.0)>

In [30]:
# pd.DataFrame(input, columns=input_col_names)
input = input[:,:]
pd.DataFrame(input, columns=input_col_names)

,qc,qr,nc,nr,pgam,lamc,lamr,n0r,rho_clubb
0,1.333108e-08,0.0,158.360965,0.0,10.111111,222222.22214,0.0,0.0,0.989029


In [31]:
input_quantile = pd.DataFrame(input_scaler.transform(input), columns=input_col_names)
input_quantile

/glade/u/home/wchuang/.local/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


,qc,qr,nc,nr,pgam,lamc,lamr,n0r,rho_clubb
0,0.0,0.0,0.0,0.0,0.911275,0.344813,0.0,0.0,0.408934


In [32]:
output_quantile = nn_tau_em.predict(input_quantile)
output_quantile

1/1 [==============================] - 0s 71ms/step


array([[0.68098989, 0.95264907, 0.94627122]])

In [33]:
output = output_scaler.inverse_transform(output_quantile)
pd.DataFrame(output, columns=output_col_names)

/glade/u/home/wchuang/.local/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


,qctend,nctend,nrtend
0,-3.878286e-09,-9.743312,37.28586


#### Checking the input quantile scaler

In [34]:
# # Read in the fortran quantile inputs
fortran_input_quantile_file = fortran_directory + "test_quantile_input.dat"
fortran_input_quantile = np.fromfile(fortran_input_quantile_file, dtype=np.float64, sep=' ').reshape(1, -1)
fortran_input_quantile
print(fortran_input_quantile[:,:])
# # Quantile transform with the python scaler
input_quantile = input_scaler.transform(pd.DataFrame(input))
print(input_quantile)
# # Compare python-quantile-transformed inputs to the fortran-imported inputs
input_quantile_same = np.allclose(fortran_input_quantile[:,:], input_quantile, atol=1e-7)
input_quantile_same
# # Are they the same?
if input_quantile_same:
    print("Arrays are equal within the specified tolerance.")
else:
    print("Arrays are not equal within the specified tolerance.")

[[0.         0.         0.         0.         0.91127528 0.34481357
  0.         0.         0.40893432]]
[[0.         0.         0.         0.         0.91127549 0.34481345
  0.         0.         0.40893432]]
Arrays are equal within the specified tolerance.


/glade/u/home/wchuang/.local/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


#### Checking the neural network, which gives us the output quantile values

In [35]:
fortran_output_quantile_file = fortran_directory + "test_quantile_output.dat"
fortran_output_quantile = np.fromfile(fortran_output_quantile_file, dtype=np.float64, sep=' ').reshape(1, -1)
print(fortran_output_quantile[:,:])
# Run the quantile transformed inputs through the python nn to get python-quantile-transformed outputs
output_quantile = nn_tau_em.predict(input_quantile)
print(output_quantile)
# Compare the python-quantile-transformed outputs to the fortran-imported-quantile-transformed outputs
output_quantile_same = np.allclose(output_quantile, fortran_output_quantile[:,:], atol=1e-7)
# Are these the same?
if output_quantile_same:
    print("Arrays are equal within the specified tolerance.")
else:
    print("Arrays are not equal within the specified tolerance.")

[[0.68099008 0.95264908 0.9462712 ]]
1/1 [==============================] - 0s 33ms/step
[[0.68098987 0.95264907 0.94627123]]
Arrays are equal within the specified tolerance.


#### Checking the output quantile scaler

In [36]:
# Reverse quantile transform the outputs using python
fortran_output_file = fortran_directory + "test_output.dat"
fortran_output = np.fromfile(fortran_output_file, dtype=np.float64, sep=' ').reshape(1, -1)
print(fortran_output[:,:])
# Compare the python outputs to the fortran-imported outputs
output = output_scaler.inverse_transform(output_quantile)
print(output)
# Are they the same?
output_same = np.allclose(output, fortran_output[:,:], atol=1e-7)
output_same

[[-3.87827876e-09 -9.74331043e+00  3.72858520e+01]]
[[-3.87828669e-09 -9.74331192e+00  3.72858607e+01]]


/glade/u/home/wchuang/.local/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


True

#### If all checks pass here then the fortran neural network has been implemented successfully
